## 수도권, 지방권 아파트 월별 평균 매매 금액 (2014년 ~ 2024년)

### 팀 AWS db 에 점근

## 데이터 가져오기

- 원하는 화면이 나올때 까지 selenium 으로 진행 

- 그 후 데이터들을 beautifulsoup 으로 가져오기

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time

options = webdriver.ChromeOptions()

url = 'https://www.reb.or.kr/r-one/portal/stat/easyStatPage.do'  # 부동산 통계정보 사이트 url
small_driver = webdriver.Chrome(service=Service('/home/yoon/ws/eda/driver/chromedriver'),
                          options=options)

small_driver.get(url)

small_driver.maximize_window()  # 전체화면 크기로 지정

driver = small_driver

# '전국주택가격동향조사' 클릭하기
survey_btn = driver.find_element(By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/span/a/em')
survey_btn.click()

# '월간동향' 클릭하기
montly_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/span')))
montly_btn.click()

# '아파트' 클릭하기
apart_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/span')))
apart_btn.click()

# '매매가격' 클릭하기
price_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/ul/li[5]/span/a/em')))
price_btn.click()

# '(월) 평균매매가격_아파트' 클릭하기
montly_price_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/ul/li[5]/ul/li[1]/span/a')))
montly_price_btn.click()

# 유의사항 창 닫기 클릭하기
close_warning_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="SP1A"]/div/div[1]/button')))
close_warning_btn.click()

# 콤보 박스 가져와서 Select 클래스로 감싸기
class_combo_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="treeClsOptCbo"]')))
select_class = Select(class_combo_box)

each_classes = class_combo_box.find_elements(By.TAG_NAME, 'option')
for idx, each in enumerate(each_classes):
    if idx == 1:  
        select_class.select_by_value(each.get_attribute('value'))
time.sleep(0.2)

# '분류선택' 에서 모든 체크 해재 클릭하기
reset_check_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="clsBox"]/div[2]/span[2]/button[2]')))
reset_check_btn.click()

# 수도권, 지방권만 클릭하기
capital_check = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeClsData"]/ul/li[2]/span/span[2]')))  # 수도권 체크 박스
capital_check.click()
noncapital_check = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeClsData"]/ul/li[3]/span/span[2]')))  # 지방권 체크 박스
noncapital_check.click()

# '검색기간' 체크
period_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wrttimeTypeB"]')))
period_btn.click()

# value 값이 2014인 키를 보낸다. (2014년 선택하기)
period_combo_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wrttimeStartYear"]')))
combo_box_list = period_combo_box.find_elements(By.TAG_NAME, 'option')

for idx, year in enumerate(combo_box_list):
    if idx == 2:
        period_combo_box.send_keys(year.get_attribute('value'))


# '조회' 클릭하기
search_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="addStatTab"]')))
search_btn.click()

# IBSheet 경고창 해제
try:
    alert = WebDriverWait(driver, 10).until(EC.alert_is_present())
    alert.accept()  # 경고 창 닫기
except:
    print("경고 창이 나타나지 않았습니다.")

# beautifulsouop 으로 현재 페이지 html 형태로 가져오기
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')


## beautifulsoup 으로 데이터 가져오기

- YEAR : 연도 (int)

- MONTH : 월 (int)

- CATEGORY : 수도권 or 지방권 (varchar(8))

- PRICE : 해당 기간 아파트 평균 매매가 (int)

In [3]:
sql_insert_query = """
INSERT INTO APARTMENT_AVG_PRICE_CATEGORY_MONTHLY (YEAR, MONTH, CATEGORY, PRICE)
VALUES (%s, %s, %s, %s)
"""

place_name_panel = soup.find_all('div', class_='IBPageOne')[0]
place_name_list = place_name_panel.find_all('tr')[1:]

# 각 category 이름 먼저 가져오기
for idx, category in enumerate(place_name_list):
    category_name = category.find('div').get_text()  # 카테고리 이름

    # 가격 데이터
    price_panel = soup.find_all('div', class_='IBPageOne')[1]  # 표
    price_list = price_panel.find_all('tr')[1:]  # 표의 두 행

    # 날짜 데이터
    date_panel = soup.find('div', class_='IBHeadMid')  # 기간이 들어있는 판넬
    date_list = date_panel.find_all('tr')[1]
    each_date = date_list.find_all('td')[1:129]  # 1인덱스 부터 128번 인덱스까지 가져오기 

    # category 별 가격 데이터 가져오기
    category_price_list = price_list[idx].find_all('td')[1:129]  # 수도권 선택

    for date_idx, each_date in enumerate(each_date):
        date = each_date.get_text()  # 날짜
        year = int(date[:4])  # 년도
        if len(date) == 8:
            month = int(date[6])  # 월
        else:
            month = int(date[6:8])  # 월

        # 해당 날짜의 가격 가져오기
        each_price = category_price_list[date_idx]
        ori_price = each_price.get_text()
        slice_price = int(ori_price[:7].replace(',', ''))
        real_price = slice_price * 1000  # 억 단위로 나타내기
        print(f'{category_name}의 {year}년 {month}월 아파트 평균 매매가 = {real_price}')
        cur.execute(sql_insert_query, (year, month, category_name, real_price))

remote.commit()
cur.close()
remote.close()

수도권의 2014년 1월 아파트 평균 매매가 = 336244000
수도권의 2014년 2월 아파트 평균 매매가 = 337285000
수도권의 2014년 3월 아파트 평균 매매가 = 338751000
수도권의 2014년 4월 아파트 평균 매매가 = 338822000
수도권의 2014년 5월 아파트 평균 매매가 = 338573000
수도권의 2014년 6월 아파트 평균 매매가 = 338393000
수도권의 2014년 7월 아파트 평균 매매가 = 338403000
수도권의 2014년 8월 아파트 평균 매매가 = 338864000
수도권의 2014년 9월 아파트 평균 매매가 = 340449000
수도권의 2014년 10월 아파트 평균 매매가 = 341834000
수도권의 2014년 11월 아파트 평균 매매가 = 342503000
수도권의 2014년 12월 아파트 평균 매매가 = 342794000
수도권의 2015년 1월 아파트 평균 매매가 = 341618000
수도권의 2015년 2월 아파트 평균 매매가 = 342811000
수도권의 2015년 3월 아파트 평균 매매가 = 344602000
수도권의 2015년 4월 아파트 평균 매매가 = 347176000
수도권의 2015년 5월 아파트 평균 매매가 = 349354000
수도권의 2015년 6월 아파트 평균 매매가 = 351598000
수도권의 2015년 7월 아파트 평균 매매가 = 365184000
수도권의 2015년 8월 아파트 평균 매매가 = 366869000
수도권의 2015년 9월 아파트 평균 매매가 = 368793000
수도권의 2015년 10월 아파트 평균 매매가 = 370838000
수도권의 2015년 11월 아파트 평균 매매가 = 372256000
수도권의 2015년 12월 아파트 평균 매매가 = 373370000
수도권의 2016년 1월 아파트 평균 매매가 = 373471000
수도권의 2016년 2월 아파트 평균 매매가 = 373167000
수도권의 2016년 3월 아파트 평균 매매가 = 37308

In [4]:
driver.close()
cur.close()
remote.close()